# Similarities 

The aim of this notebook is to calculate similarity scores and save them down to JSON.
As it's a heavy job, it will be run separately in Google Colab.

## Imports

In [3]:
# add root path to system path
import sys
sys.path.append('../')

In [11]:
import itertools
import pandas as pd
import json
import os
from tqdm import tqdm
from scripts.helpers import get_probabilities_from_json
import numpy as np

In [7]:
root_path = os.path.dirname(os.path.abspath(""))

## Data

Note that data preprocessing has been done in [preprocess.ipynb](preprocess.ipynb). The notebook will clean, merge and save the data to `data/processed`.

In [8]:
movie_df = pd.read_csv(os.path.join(root_path, 'data' , 'processed' ,'preprocessed.csv'), index_col=0)
print(movie_df.shape)
movie_df.head(5)

(22394, 10)


,name,release_year,rating,num_votes,wikipedia_id,plot,freebase_id,languages,countries,genres
0,Miss Jerry,1894.0,5.3,207,10109752,After finding out that her father is suffering...,/m/02q23xk,['Silent film'],['United States of America'],"['Short Film', 'Silent film', 'Indie', 'Black-..."
1,The Corbett-Fitzsimmons Fight,1897.0,5.3,484,28703057,The film no longer exists in its entirety; how...,/m/0czdh_n,[],[],['Sports']
2,The Story of the Kelly Gang,1906.0,6.0,855,142995,The Story of the Kelly Gangs tone is of sorrow...,/m/0120y4,['English Language'],['Australia'],"['Crime Fiction', 'Silent film', 'Biography', ..."
3,Robbery Under Arms,1907.0,4.3,25,32986669,Key scenes of the film included the branding o...,/m/04p7yxx,['Silent film'],['Australia'],"['Silent film', 'Drama']"
4,"Captain Midnight, the Bush King",1911.0,5.4,18,32987200,Edgar Dalimore is the son of wealthy station o...,/m/0h569x9,['Silent film'],[],['Silent film']


## Embeddings

Since we're already got the genre probabilities for the movies using Google Colab and downloaded them (see Readme), we can simply read the json.

In [17]:
DECADE = 1920  # change the decade whenever necessary
probabilities = get_probabilities_from_json(DECADE)

### Modelling

### Similarity Matrix

Once we have vectorized the plots, we can calculate the similarities between the plots with dot product.

In [18]:
# generate all possible combinations for dot product 
movie_sample = list(probabilities.keys())
combinations = list(itertools.combinations(movie_sample, 2))

In [27]:
similarity_dict = {}

def get_vector(movie):
    labels = movie['genres']['labels'] + movie['themes']['labels']
    scores = movie['genres']['scores'] + movie['themes']['scores']
    
    # sort the labels and reorder the scores accordingly
    labels, scores = zip(*sorted(zip(labels, scores)))
    return np.array(scores)

for movie_1_id, movie_2_id in tqdm(combinations):
    movie_1 = probabilities[movie_1_id]
    movie_2 = probabilities[movie_2_id]

    vector_1 = get_vector(movie_1)
    vector_2 = get_vector(movie_2)

    similarity = np.dot(vector_1, vector_2)
    similarity_dict['-'.join([movie_1_id, movie_2_id])] = similarity    

100%|██████████| 85078/85078 [00:00<00:00, 118146.40it/s]


In [28]:
with open(os.path.join(root_path, 'data', 'classification', 'similarities', f'similarities_{DECADE}s.json'), "w") as outfile: 
    json.dump(similarity_dict, outfile)